In [1]:
#libs import
from IPython.display import Audio
from scipy.io import wavfile

#read audiofile, get sample rate and audio array
samplerate, audio_file = wavfile.read('bethoven.wav')
#setting start and end of the audiofile for playing
start = samplerate * 14 
end = start + samplerate * 10 
# run player
Audio(data= audio_file[start:end, 0], rate=samplerate)

# Метод главных компонент (principal component analysis, PCA)
Суть метода основывается на сжатии с потерями: аудиофайл необходимо представить таким образом, чтобы он занимал наименьшее количество места на диске и при этом человеческий слух не замечал бы резкого снижения качества.  
Объяснить принцип работы PCA можно, если представить звук как набор неких признаков, по которым формируется аудиосигнал. Эти признаки обобщаются в новый набор признаков, по статическим характеристикам наиболее близкий к оригинальному набору. 


Алгоритм использования PCA таков: 
Изменить форму сигнала до 1024 коломн. Запустить PCA и снизить размерность до нового числа коломн. В данном случае коломны это признаки и их уменьшение приведет к потере качества звука. 

In [2]:
from sklearn.decomposition import PCA
import numpy as np

"""
    Compression Function using PCA
    signal - signal that is needed to be compressed
    n_components - number of components to keep & equals min(n_samples, n_features)
    block_size - features size
"""
def pca_reduce(signal, n_components, block_size=1024):
    # number of samples
    samples = len(signal)
    # getting value of how many zeros is needed to get block_size  
    hanging = block_size - np.mod(samples, block_size)
    # padding signal by adding zeros to the end
    padded = np.lib.pad(signal, (0, hanging), 'constant', constant_values=0)
    # reshaping signal
    reshaped = padded.reshape((len(padded) // block_size, block_size))
    # running PCA
    pca = PCA(n_components=n_components)
    # make array with number of features = block_size 
    pca.fit(reshaped)
    # using PCA to the reshaped signal
    transformed = pca.transform(reshaped) 
    # transform data back to original space
    reconstructed = pca.inverse_transform(transformed).reshape((len(padded)))
    return pca, transformed, reconstructed

In [22]:
pca, transformed, reconstructed = pca_reduce(audio_file[start:end, 0], 50, 4096)
Audio(data= reconstructed, rate=samplerate)